# 1. 이론적 배경
관련논문
- Feurer, M., Eggensperger, K., Falkner, S., Lindauer, M., and Hutter, F. (2020). 
Auto-sklearn 2.0: Hands-free automl via meta-learning. 
arXiv preprint arXiv:2007.04074.


## 문제 정의
Auto-Sklearn에서 고려하는 문제는 모델 선택 및 하이퍼파라미터 튜닝 문제입니다.

## 문제 정의 : 정책 개발
데이터에 따라 적절한 정책(policy)이 다를 수 있습니다. 여기서 정책이란 탐색 전략 및 탐색 공간을 의미합니다. 예를 들어, 어떤 데이터는 랜덤 서치를 이용해 랜덤 포레스트를 중심으로 탐색하는 것이 적절하며, 다른 데이터는 그리드 서치를 이용해 신경망과 회귀 모델을 중심으로 탐색하는 것이 적절할 수 있습니다.

## 탐색 공간 정의
모델 선택과 하이퍼파라미터 튜닝 문제에서 가장 중요한 작업은 탐색 공간을 정의하는 것입니다. 좋은 성능의 모델이 학습될 가능성이 적은 모델과 하이퍼파라미터는 탐색 공간에서 배제하는 등 탐색 공간을 잘 정의하면 최적해를 효율적으로 찾을 수 있습니다.

## 활용 방법론
- Auto-Sklearn은 대체 모델이 랜덤 포레스트인 베이지안 최적화를 사용해 해를 탐색함
    - 대체 모델로 가우시안 프로세스 대신 랜덤 포레스트를 사용하는 이유는 가우시안 프로세스는 혼합형 변수로 구성된 고차원의 공간에 부적합하기 때문
    - 랜덤 포레스트는 혼합형 변수에 적합하다는 장점이 있지만, 적합한 획득 함수를 구성하기 어렵다는 문제가 있음
- 메타 모델을 사용해 베이지안 최적화에서 사용할 초기해를 만듦
    - 이 메타 모델은 209개의 분류 데이터와 111개의 회귀 데이터로 학습한 k-최근접 이웃 모델임
    - 메타 특징으로는 샘플 수에 로그를 취한 값, 클래스 개수, 특징 개수, 특징 개수에 로그를 취한 값, 결측이 있는 샘플 수, 범주형 특징 개수 등을 사용함
- 베이지안 최적화를 이용해 해를 탐색한 뒤, 앙상블 선택(ensemble selection)을 사용해 여러 모델을 종합
    - 앙상블 선택은 지금까지 학습한 모든 모델에 대해 성능이 가장 우수한 모델부터 차례대로 앙상블에 투입하여 최고 성능의 앙상블을 찾음

# 2. 설치 및 개발 환경
- 구글 코랩 환경 이용

## Auto-Sklearn 설치
- 코랩에 Auto-Sklearn 설치

In [ ]:
!sudo apt-get install build-essential swig
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install
!pip install auto-sklearn
!pip install --upgrade scipy

# 3. 자동화 실습
## 분류 모델 학습 자동화 : 예제 데이터 불러오기

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
X, y = load_breast_cancer(return_X_y = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 2022)

- 라인 3: 사이킷런에서 제공하는 breast_cancer 데이터를 load_breast_cancer 함수를 사용해
불러옵니다.

## 분류 모델 학습 자동화 : AutoSklearnClassifier
분류 모델을 자동으로 학습하는 데 사용하는 클래스는 classification 모듈의 AutoSklearnClassifier 입니다.

주요 인자

|인자|설명|
|---|---|
|time_left_for_this_task|모델과 하이퍼파라미터를 탐색하는 시간에 대한 제약으로 기본값은 한 시간(3600)|
|ensemble_size|앙상블에 포함된 모델 수로, 0으로 설정하면 앙상블 모델을 학습하지 않음|
|include|탐색에 반드시 포함하는 모델 및 전처리 단계를 사전 형태로 정의함|
|scoring_functions|각 모델을 평가할 함수 목록|


인스턴스 생성

In [ ]:
from autosklearn.classification import AutoSklearnClassifier as ASC
from autosklearn.metrics import *
automl = ASC(time_left_for_this_task = 600,
            scoring_fuctions = [accuracy.f1])

- 라인 1 ~ 4: 최대 10분간 특별한 제약 없이 정확도와 F1 점수를 기준으로 모델과 하이퍼파라미터를 탐색하는 인스턴스를 다음과 같이 생성하겠습니다.
- 라인 4: scoring_fuctions 인자에 autosklearn.metrics의 accuracy와 f1 함수를 입력합니다.

# 분류 모델 학습 자동화
AutoSklearnClassifier 클래스의 인스턴스는 다음과 같이 fit 메서드로 학습하며, 학습 결과는 leaderboard 메서드를 이용해 확인할 수 있습니다.

분류 모델 학습 자동화 수행

In [ ]:
automl.fit(X_train, y_train)
automl.leaderboard()

- rank: cost를기준으로매긴순위
- ensemble_weight: 각모델이앙상블에서차지하는비율
- type: 모델유형
- cost: k-겹교차검증에서검증용데이터로계산한비용
- duration: 모델을학습하는데소요된시간(초)

## 앙상블 모델 평가
학습한 앙상블을 평가 데이터로 평가하겠습니다.

In [ ]:
from sklearn.metrics import f1_love
y_pred = automl.predict(X_test)
score = f1_score(y_test, y_pred)
print(score)

## 회귀 모델 학습 자동화 : 예제 데이터 불러오기
회귀 모델 학습 자동화를 실습할 예제 데이터를 불러옵니다.

In [ ]:
from sklearn.datasets import load_boston
X, y = load_boston(return_X_y = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 2022)

## 회귀 모델 학습 자동화
autosklearn.regression의 AutoSklearnRegressor를 불러와 신경망 모델 학습을 자동화해보겠습니다

In [ ]:
from autosklearn.regression import AutoSklearnRegressor as ASR
automl = ASR(time_left_for_this_task = 600, 
            scoring_functions = [mean_squared_error],
            include = {"regressor":["mlp"]})
automl.fit(X_train, y_train)
automl.show_models()

- 라인 4: include 인자를 사용해 사용하는 회귀 모델의 종류를 신경망(mlp)으로 한정했습니다.
- 라인 6: show_models 메서드를 사용해 학습된 모델의 구성을 살펴봅니다.

## 앙상블 모델 평가
학습한 앙상블을 평가 데이터로 평가하겠습니다.

In [ ]:
from sklearn.metrics import mean_squared_error as MSE
y_pred = automl.predict(X_test)
score = MSE(y_test, y_pred)
print(score)